In [2]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
import matplotlib.pyplot as plt
from keras_cv.models import StableDiffusion
from keras_cv.models.stable_diffusion.diffusion_model import DiffusionModel
import tensorflow.keras as keras

2.12.0
You do not have pycocotools installed, so KerasCV pycoco metrics are not available. Please run `pip install pycocotools`.
You do not have pyococotools installed, so the `PyCOCOCallback` API is not available.
You do not have Waymo Open Dataset installed, so KerasCV Waymo metrics are not available.


/users/skoka/.venv/newMLEnv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tf.__version__

'2.12.0'

In [4]:
Stable_diffusion = StableDiffusion(img_height=512, img_width=512)

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE


In [5]:
decoder = Stable_diffusion.decoder

2023-04-26 23:51:23.545060: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79085 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:47:00.0, compute capability: 8.0


In [6]:
diffusion_model = DiffusionModel(img_width = 512, img_height = 512, max_text_length=0)

In [7]:
diffusion_model.summary()

Model: "diffusion_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 320)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 1280)         410880      ['input_3[0][0]']                
                                                                                                  
 input_4 (InputLayer)           [(None, 64, 64, 4)]  0           []                               
                                                                                                  
 activation_1 (Activation)      (None, 1280)         0           ['dense[0][0]']                  
                                                                                    

In [12]:
diffusion_model.input_shape

[(None, 64, 64, 4), (None, 320), (None, 0, 768)]

In [17]:
import math
def get_timestep_embedding(timestep, batch_size, dim=320, max_period=10000):
        half = dim // 2
        freqs = tf.math.exp(
            -math.log(max_period) * tf.range(0, half, dtype=tf.float32) / half
        )
        args = tf.convert_to_tensor([timestep], dtype=tf.float32) * freqs
        embedding = tf.concat([tf.math.cos(args), tf.math.sin(args)], 0)
        embedding = tf.reshape(embedding, [1, -1])
        return tf.repeat(embedding, batch_size, axis=0)

In [18]:
# we are going to need to modify this code a lot 
def generate_pattern(layer_name, filter_index, size=64):
    # Build a model that outputs the activation
    # of the nth filter of the layer considered.
    layer_output = diffusion_model.get_layer(layer_name).output
    # Isolate the output 
    new_model = tf.keras.models.Model(inputs=diffusion_model.inputs, outputs=layer_output)
    
    # We start from a gray image with some uniform noise
    input_img_data = np.random.random((1, size, size, 4)) * 20 + 128.
    I = tf.Variable(input_img_data, name='image_var', dtype = 'float64')
    #I = preprocess_input(I_start) # only process once
    # Run gradient ascent for 40 steps
    eta = 1
    for i in range(50):
        
        with tf.GradientTape(watch_accessed_variables=False) as tape:
            time_embedding = get_timestep_embedding(i, 1)
            # create a tensor with the following size of all zeros: (None, 0, 768)
            word_embedding = tf.zeros((1, 0, 768))
            tape.watch(I)
            # get variable to maximize
            model_vals = new_model((I, time_embedding, word_embedding))
            loss = tf.reduce_mean(model_vals[:, :, :, filter_index])

        # Compute the gradient of the input picture w.r.t. this loss
        # add this operation input to maximize
        grad_fn = tape.gradient(loss, I)
        # Normalization trick: we normalize the gradient
        grad_fn /= (tf.sqrt(tf.reduce_mean(tf.square(grad_fn))) + 1e-5) # mean L2 norm
        I = I - (grad_fn * eta) # one iteration of maximizing
    # decode the resulting input image
    I = decoder(I)
    # return the numpy matrix so we can visualize 
    img = I.numpy()
    return img

In [19]:
img_output = generate_pattern('padded_conv2d_47', 5)

2023-04-26 23:57:42.863654: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:417] Loaded runtime CuDNN library: 8.2.4 but source was compiled with: 8.6.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2023-04-26 23:57:42.865674: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at conv_ops.cc:1068 : UNIMPLEMENTED: DNN library is not found.


UnimplementedError: Exception encountered when calling layer 'conv2d_38' (type Conv2D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} DNN library is not found. [Op:Conv2D]

Call arguments received by layer 'conv2d_38' (type Conv2D):
  • inputs=tf.Tensor(shape=(1, 66, 66, 4), dtype=float32)

In [51]:
# plot the image
img_output = (img_output * 127.5) + 127.5
plt.imshow(img_output[0, :, :, 0].astype('int32'), cmap='viridis')